In [1]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os
from bs4 import BeautifulSoup
import re
from datetime import datetime



In [22]:
def prepros(html) :
    d=json.loads(html)
    
    soup = BeautifulSoup(d['result_list_box_html'],'lxml')
    
    # this selects the main part of the html
    tabel = soup.find('div', attrs={'class':'results component--default'})
    
    # Jobindex contain both own postings and postings from other
    # the two are treated differently in the data
    # Jobindex' imported postings
    others = re.compile('r[\d]+')
    tabel1 = tabel.findAll('div', attrs={'data-beacon-tid':others})
    #print(tabel1)

    # Jobindex' own postings
    own = re.compile('h[\d]+')
    tabel2 = tabel.findAll('div', attrs={'data-beacon-tid':own})
    #print(tabel2)
    
    return tabel, tabel1, tabel2

def job_descr_own(tabel2) : # den er ikke færdig: tag <ul> og saml elementer, der hører sammen
    desc_own = []

    # tag ul skal med 
    for l in tabel2 :
        g = l.findAll('p')
        for m in g :
            desc_own.append(m.text)
    return desc_own

def job_descr_oth(tabel1) : # færdig og virker
    desc_oth = []
    p=re.compile('"')
    lineshift = re.compile('\n')
    besk = []
    for t_ in tabel1 :
        besk.append(lineshift.sub("", t_.text))
    for t in besk :
        desc_oth.append(t.split(sep='    ')[1])
    return desc_oth
    
def job_title_oth (tabel) : # færdig og virker
    # udled jobs andre
    jobs_oth = []
    j = tabel.findAll('strong') 
    for l in j :
        jobs_oth.append(l.text)
    return jobs_oth


def firm_place(tabel) : # tjek den igen, der kommer lidt for mange elementer ud
    firm_city=tabel.find_all('b')
    #print(firm_city)
    firm=[]
    city=[]
    cc=0
    for i in firm_city:
        if cc %2==0:
            firm.append(i.text)
        else :
            city.append(i.text)
        cc += 1
    return firm, city


def dates(tabel) : # finds only dates for others; needs also dates for own
    dato_site=tabel.find_all('cite')

    website=[]
    for i in dato_site:
        s_txt=i.text
        site=s_txt.split(",")[0]
        website.append(site)

    indented=[]
    for i in dato_site:
        d_txt=i.text
        dates=d_txt.split(",")
        if dates[0] == "StepStone":    # Har behov for at blive fixet
            indented.append(indented[-1])
        else:
            indented.append(dates[1])

    monthval={'januar':'01','februar':'02', 'marts':'03', 'april':'04', 'maj':'05', 'juni':'06',
              'juli':'07','august':'08','september':'09','oktober':'10','november':'11','december':'12'}
    indented_d=[]

    for i in indented:
        datotal=(int(i.replace(i[3:-4], monthval.get(i[5:-5]))))
        indented_d.append(datetime.strptime(str(datotal), '%d%m%Y'))
    return indented_d



#hyper = []
#x1 = tabel.findAll('a', attrs={'href':True}) 
#for x in x1 :
#    hyper.append(x.)
#    
#print(x1)



In [23]:
def process_data() :
    base_path = r"C:\Notebooks\group19\Eksamensprojekt/jobindex.txt" # file with scraped jobindex data

    f = open(base_path, 'r') # open the file for reading
    
    # loop through the file one line at a time
    for html in f :
        # here the various processing functions will be called
        tabel, tabel1, tabel2= prepros(html)
        
        desc_own = job_descr_own(tabel2)
        desc_oth = job_descr_oth(tabel1)
        jobs_oth = job_title_oth(tabel)
        firm, city = firm_place(tabel)
        post_dates = dates(tabel)
        
        # der skal laves en zip-funktion, så de kan blive sat rigtig sammen
    return desc_own, desc_oth, jobs_oth, firm, city, post_dates
    f.close()

desc_own, desc_oth, jobs_oth, firm, city, post_dates = process_data()
print(len(desc_own))
print(len(desc_oth))
print(len(jobs_oth))
print(len(firm))
print(len(city))
print(len(post_dates))

49
6
6
22
21
6
